In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import  preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold,StratifiedKFold
import matplotlib.pyplot as plt

In [2]:
#try xgboost
#fucntion from SRK
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, \
     seed_val=0, early_stop = 20,num_rounds=10000, eta = 0.1,\
     max_depth = 6,cv_dict = None,verbose_eval=True):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = eta
    param['max_depth'] = max_depth
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y,feature_names=feature_names)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y,feature_names=feature_names)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist,\
        early_stopping_rounds=early_stop,evals_result = cv_dict,verbose_eval = verbose_eval)
    else:
        xgtest = xgb.DMatrix(test_X,feature_names=feature_names)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

class CVstatistics(object):
    
    """
    self.result : the result dataframe storing the cv results
    self.endpoint : the first ending point for the validations
    self.turns: the turns for each validation
    
    validCurve : plot the validation curve,stop at the first endpoint
    """
    def __init__(self,result_dict,metric,k=5):
        self.metric = metric
        if type(result_dict) == pd.DataFrame:
            self.result = result_dict
        else:
            tempDict = {}
            for phase in ['train','test']:
                for turn in range(k):
                    tempDict[phase+str(turn)]=cv_result[turn][phase][metric]
                    self.result=pd.DataFrame(dict([ (key,pd.Series(v)) for key,v in tempDict.iteritems()]))    
        
        self.endpoint =len(self.result.filter(like = 'train').dropna())
        
        self.turns = self.result.filter(like = 'test').\
            apply(lambda x : ~np.isnan(x)).cumsum(axis=0).iloc[len(self.result)-1,:]

    def validCurve(self,start=0,stop_at_first = True):
        if stop_at_first:
            eout = self.result.filter(like = 'test').dropna().mean(axis=1)
            ein =  self.result.filter(like = 'train').dropna().mean(axis=1)
        else:
            eout = self.result.filter(like = 'test').mean(axis=1)
            ein =  self.result.filter(like = 'train').mean(axis=1)
        plt.plot(range(len(eout)), eout,
        range(len(ein)), ein)
        plt.xlabel("turn")
        plt.ylabel(self.metric)
        plt.title('Validation Curve')
        
        plt.show()
    
    def errorsAt(self,turn):
        eout = self.result.filter(like = 'test').loc[turn].mean()
        ein = self.result.filter(like = 'train').loc[turn].mean()
        return eout,ein
    

def showImportance(model,factor_name):
    factors = model.get_score(importance_type=factor_name)
    factor_list = []
    total = sum(factors.values())
    for key in factors:
        factors[key] = factors[key]*1.0/total
        factor_list.append((key,factors[key]))
    return sorted(factor_list,key=lambda x : x[1],reverse=True)
    


In [3]:
#from "this is a lit"s python version by rakhlin
def singleValueConvert(df1,df2,column,minimum_size=5):
    ps = df1[column].append(df2[column])
    grouped = ps.groupby(ps).size().to_frame().rename(columns={0: "size"})
    df1.loc[df1.join(grouped, on=column, how="left")["size"] <= minimum_size, column] = -1
    df2.loc[df2.join(grouped, on=column, how="left")["size"] <= minimum_size, column] = -1
    return df1, df2

In [15]:
#new feature : manager base distance and manager action scope
#new training features: 'm_distance','m_m_distance'
#also consideing adding 'mlat','mlon'
def manager_statistics(train_df,test_df,update_df =None,random = None):
    
    #adding the features about distance and location
    temp=train_df[['manager_id',"latitude", "longitude"]].dropna()
    mean_value = temp.groupby('manager_id')[["latitude", "longitude"]].mean().round(4)
    mean_value.columns = ['mlat','mlon']
    std_value = train_df.groupby('manager_id')[["latitude", "longitude"]].std()
    mstd = std_value[["latitude", "longitude"]].mean()
    std_value['latitude']=std_value['latitude'].fillna(mstd['latitude'])
    std_value['longitude']=std_value['longitude'].fillna(mstd['longitude'])
    #manager mean distance
    std_value['m_m_distance'] = map(lambda x,y:np.sqrt(x**2+y**2).round(4),\
                                    std_value['latitude'],std_value['longitude'])
    
    if random:
        std_value['m_m_distance'] *= np.random.uniform(1 - random, 1 + random, len(std_value))
        mean_value['mlat'] *= np.random.uniform(1 - random, 1 + random, len(mean_value))
        mean_value['mlon'] *= np.random.uniform(1 - random, 1 + random, len(mean_value))

    updateM = test_df[['manager_id']].join(mean_value, on = 'manager_id', how="left")[['mlat','mlon']]
    updateD = test_df[['manager_id']].join(std_value, on='manager_id', how="left")['m_m_distance']
    
    if update_df is None: update_df = test_df
    for f in ['mlat','mlon','m_m_distance']:
        if f not in update_df.columns: 
            update_df[f] = np.nan
    
    update_df.update(updateD)
    update_df.update(updateM)
    
    #adding the features about other things
    other_feature = ['bathrooms','bedrooms','price',"price_per_bed","bath_per_bed",\
                     "price_per_room",'num_photos','num_features','num_description_words']
    mean_value = train_df.groupby('manager_id')[other_feature].mean()
    mean_value = mean_value.add_prefix('m_mean_')
#    std_value = train_df.groupby('manager_id')[other_feature].mean()
#    std_value = std_value.add_prefix('m_std_')
    
    new_mean_feature = list(mean_value.columns)
#    new_std_feature = list(std_value.columns)
    
    if random:
        for f in new_mean_feature:
            mean_value[f] *= np.random.uniform(1 - random, 1 + random, len(mean_value))
#        for f in new_std_feature:
#            std_value[f] *= np.random.uniform(1 - random, 1 + random, len(mean_value))
    
    updateM = test_df[['manager_id']].join(mean_value, on = 'manager_id', how="left")[new_mean_feature]
#    updateD = test_df[['manager_id']].join(std_value, on = 'manager_id', how="left")[new_std_feature]
    
    for f in new_mean_feature:
        if f not in update_df.columns: 
            update_df[f] = np.nan
            
#    for f in new_std_feature:
#        if f not in update_df.columns: 
#            update_df[f] = np.nan
    
    update_df.update(updateM)
#    update_df.update(updateD)

    
    """
    TODO CONSIDER USING THESE TO ADD RANDOM INTO THE VALUES
    # Add uniform noise. Not mentioned in original paper.adding to each manager
    if random:
        temp[new_feature] *= np.random.uniform(1 - random, 1 + random, len(temp))     
    """

In [5]:
#lodaing data
data_path = "../../kaggleData/2sigma/"
train_file = data_path + "train.json"
test_file = data_path + "test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)


(49352, 15)
(74659, 14)


In [6]:
#basic numerical features
features_to_use  = ["bathrooms", "bedrooms", "latitude", "longitude", "price"]

In [7]:
#some transfromed features
# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)

# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))

# convert the created column to datetime object so as to extract more features 
train_df["created"] = pd.to_datetime(train_df["created"])

# Let us extract some features like year, month, day, hour from date columns #
train_df["created_year"] = train_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour

#some new numerical features related to the price
train_df["price_per_bath"] =  (train_df["price"]*1.0/train_df["bathrooms"]).replace(np.Inf,-1)
train_df["price_per_bed"] = (train_df["price"]*1.0/train_df["bedrooms"]).replace(np.Inf,-1)
train_df["bath_per_bed"] = (train_df["bathrooms"]*1.0/train_df["bedrooms"]).replace(np.Inf,-1)
train_df["price_per_room"] = (train_df["price"]*1.0/(train_df["bedrooms"]+train_df["bathrooms"])).replace(np.Inf,-1)

# adding all these new features to use list # "listing_id",
features_to_use.extend(["num_photos", "num_features", "num_description_words",\
                        "created_year","listing_id", "created_month", "created_day", "created_hour"])
#price new features
features_to_use.extend(["price_per_bed","bath_per_bed","price_per_room"])


In [8]:
train_df,test_df  = singleValueConvert(train_df,test_df,'manager_id',1)

In [16]:
features = list(features_to_use)
categorical = ["display_address", "street_address",'building_id','manager_id']
features.extend(categorical)
#new features
features.extend(['m_m_distance','mlon','mlat'])
features.extend(['m_mean_bathrooms','m_mean_bedrooms','m_mean_price','m_mean_price_per_bed',\
                 'm_mean_bath_per_bed','m_mean_price_per_room','m_mean_num_photos',\
                 'm_mean_num_features','m_mean_num_description_words'])

#features.extend(['m_std_bathrooms','m_std_bedrooms','m_std_price','m_std_price_per_bed',\
#                 'm_std_bath_per_bed','m_std_price_per_room','m_std_num_photos','m_std_num_features',\
#                 'm_std_num_description_words'])

In [10]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 42)

In [17]:
#running and getting the cv from xgboost
cv_scores = []
cv_result = []

#K-FOLD already defined.If not ,use
#KF=KFold(len(train_X),5,shuffle=True,random_state = 42)
i=0
for dev_index, val_index in KF:
        result_dict = {}
                
        #split the orginal train set into dev_set and val_set
        dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
        
        """some preprocessing like feature constructed in cv manners"""
        skf=StratifiedKFold(dev_set['interest_level'],5,shuffle=True,random_state = 42)
        #dev set adding manager skill
        for train,test in skf:
            manager_statistics(dev_set.iloc[train,:],dev_set.iloc[test,:],\
                              update_df = dev_set,random = 0.01)
        manager_statistics(dev_set,val_set,random = 0.01)
        
        for f in categorical:
            if dev_set[f].dtype=='object':
                #print(f)
                lbl = preprocessing.LabelEncoder()
                lbl.fit(list(dev_set[f])+list(val_set[f]))
                dev_set[f] = lbl.transform(list(dev_set[f].values))
                val_set[f] = lbl.transform(list(val_set[f].values))
        
        dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        
        """ 
         runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, \
         seed_val=0, early_stop = 20,num_rounds=10000, eta = 0.1,\
         max_depth = 6,cv_dict = None):
         """
        
        preds, model = runXGB(dev_X, dev_y, val_X, val_y,early_stop  = 20,\
                              feature_names = features,cv_dict = result_dict,verbose_eval=False)
        loss = log_loss(val_y, preds)
        cv_scores.append(loss)
        cv_result.append(result_dict)
        i+=1
        print 'loss for the turn '+str(i)+' is '+str(loss)

C:\Users\dell\Anaconda2\lib\site-packages\ipykernel\__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dell\Anaconda2\lib\site-packages\ipykernel\__main__.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dell\Anaconda2\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

loss for the turn 1 is 0.557653113271
loss for the turn 2 is 0.557015725791
loss for the turn 3 is 0.557237659152
loss for the turn 4 is 0.557753398298
loss for the turn 5 is 0.558241230907


In [13]:
#plot the validation curv
cvResult = CVstatistics(cv_result,'mlogloss')
cvResult.turns

test0    363
test1    330
test2    340
test3    375
test4    309
Name: 374, dtype: int32

In [14]:
#cvResult.validCurve(stop=False)
#some errors at certain turn to see the descending
cv_scores
np.mean(cv_scores)

0.55286647131240174

In [16]:
#show the importance of the features
showImportance(model,'gain')

[('street_address', 0.058291856175022444),
 ('manager_id', 0.057872650916717956),
 ('display_address', 0.0574671560328724),
 ('listing_id', 0.057299296708135994),
 ('price_per_bed', 0.05644364380563028),
 ('price_per_room', 0.056415387352061246),
 ('price', 0.0560102202180853),
 ('bath_per_bed', 0.055819870800841435),
 ('longitude', 0.05538169121741299),
 ('latitude', 0.05514959825741516),
 ('num_description_words', 0.055083479114731444),
 ('building_id', 0.05455107411941095),
 ('created_day', 0.054036514550484735),
 ('created_hour', 0.05123663109651239),
 ('num_photos', 0.048712457065297546),
 ('num_features', 0.04861375917735466),
 ('bedrooms', 0.04403331097706631),
 ('bathrooms', 0.03979028995119792),
 ('created_month', 0.037791112463748895)]